Notebook on simple multilayer perceptron or artificial neural network
=
More examples can be found here [MLmodels](https://github.com/FluxML/model-zoo)

In [ ]:
using Flux, Statistics
using Flux.Data: DataLoader
using Flux: onehotbatch, onecold, @epochs
using Flux.Losses: logitcrossentropy
using Base: @kwdef
using CUDA
using MLDatasets

## Data loader
For every ml model, we need data

We can either generate synthetic data or load data from file

Here, we will load data from file of MNIST image data

In [ ]:
function getdata(args, device)
    ENV["DATADEPS_ALWAYS_ACCEPT"] = "true"

    # Loading Dataset	
    xtrain, ytrain = MLDatasets.MNIST.traindata(Float32)
    xtest, ytest = MLDatasets.MNIST.testdata(Float32)
	
    # Reshape Data in order to flatten each image into a linear array
    xtrain = Flux.flatten(xtrain)
    xtest = Flux.flatten(xtest)

    # One-hot-encode the labels
    ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

    # Create DataLoaders (mini-batch iterators)
    train_loader = DataLoader((xtrain, ytrain), batchsize=args.batchsize, shuffle=true)
    test_loader = DataLoader((xtest, ytest), batchsize=args.batchsize)

    return train_loader, test_loader
end

## Build a neural network model
Next thing is creating a neural network model

Here, we will build a neural network consisting two layers each containig 32 nodes

In [ ]:
function build_model(; imgsize=(28,28,1), nclasses=10)
    return Chain( Dense(prod(imgsize), 32, relu),
                  Dense(32, nclasses))
end

## Define loss and accuracy functions
Next step is defining loss ($\mathcal{L}$) and accuracy functions

Loss function computes the accuracy by comparing true values versus machine learning predicted values based on a specified function.

Here, we used logitcrossentropy function, which is good for classification

Accuracy function computes the accuracy by comparing true values versus machine learning predicted values

In [ ]:
function loss_and_accuracy(data_loader, model, device)
    acc = 0
    ls = 0.0f0
    num = 0
    for (x, y) in data_loader
        x, y = device(x), device(y)
        ŷ = model(x)
        ls += logitcrossentropy(ŷ, y, agg=sum)
        acc += sum(onecold(ŷ) .== onecold(y))
        num +=  size(x)[end]
    end
    return ls / num, acc / num
end

## Hyperparameter tuning
Neural network has often several parameters

Tuning those parameters help improving the model performance

Values of those parameters may vary widely, so it is better to specify them

Here, we are specifying three hyperparameters with just one value; however, those values can be changed during training

In [ ]:
@kwdef mutable struct Args
    η::Float64 = 3e-4       # learning rate
    batchsize::Int = 256    # batch size
    epochs::Int = 10        # number of epochs
    use_cuda::Bool = true   # use gpu (if cuda available)
end

## Build a training function
We generated ML model using neural network, defined loss and accuracy functions, and also defined some hyperparameters to tune

Now, we need to utilize them and train the model

In [ ]:
function train(; kws...)
    args = Args(; kws...) # collect options in a struct for convenience

    if CUDA.functional() && args.use_cuda
        @info "Training on CUDA GPU"
        CUDA.allowscalar(false)
        device = gpu
    else
        @info "Training on CPU"
        device = cpu
    end

    # Create test and train dataloaders
    train_loader, test_loader = getdata(args, device)

    # Construct model
    model = build_model() |> device
    ps = Flux.params(model) # model's trainable parameters
    
    ## Optimizer
    opt = ADAM(args.η)
    
    ## Training
    for epoch in 1:args.epochs
        for (x, y) in train_loader
            x, y = device(x), device(y) # transfer data to device
            gs = gradient(() -> logitcrossentropy(model(x), y), ps) # compute gradient
            Flux.Optimise.update!(opt, ps, gs) # update parameters
        end
        
        # Report on train and test
        train_loss, train_acc = loss_and_accuracy(train_loader, model, device)
        test_loss, test_acc = loss_and_accuracy(test_loader, model, device)
        println("Epoch=$epoch")
        println("  train_loss = $train_loss, train_accuracy = $train_acc")
        println("  test_loss = $test_loss, test_accuracy = $test_acc")
    end
end


In [ ]:
### Run training 
if abspath(PROGRAM_FILE) == @__FILE__
    train()
end
# train(η=0.01) # can change hyperparameters


In [ ]:
train(η=0.01) # changing learning rate